# Titanic Modeling, Evaluation and Deployment

## CRISP-DM

In [ ]:
from IPython import display
display.Image('https://upload.wikimedia.org/wikipedia/commons/b/b9/CRISP-DM_Process_Diagram.png', width=500)

## Import relevant packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

## Load data and prepare for modeling

Data preparation for modeling (including 'pd.get_dummies') has been performed in the previous notebook. Now, data are imported again and split into training and test. Models are built on training data only and, afterwards, evaluated on (previously unseen) test data. 

Attention: Reading and writing files is slightly different between Local and Cloud. You can click on the </> icon in the upper right corner, click on _Read data_, then _Select data from project_, then _Data Asset_, locate the file, and then click "Insert code to cell". Make sure that the dataframe variable name is _df_dummies_!

In [ ]:
# Local
# df_dummies = pd.read_csv('train_dummies.csv') # use full path if notebook and file in different folders! 

# Cloud: Fetch the file
# insert to code




df_dummies = pd.read_csv(body)

In [ ]:
df_dummies.head()

In [ ]:
target = df_dummies['Survived'] # feature to be predicted
predictors = df_dummies.drop(['Survived'], axis = 1) # all other features are used as predictors

In [ ]:
predictors.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=123) # 80-20 split into training and test data

In [ ]:
# Check data balancing
y_train.value_counts()

# There is no severe skew in the class distribution. No resampling needed. 
# If you want to learn more about resampling, also check https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

## Create and evaluate classification models

Predicting whether a pasenger on the Titanic survived or not is a supervised machine learning problem. Some commonly used algorithms include decision trees, random forest and logistic regression. Once a classification model has been built, evaluation metrics are calculated and interpreted. 

### Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, dt.predict(X_test)) # yields count of true negatives, false positives, false negatives, true positives

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, dt.predict(X_test)).ravel() # check that tp, fp, tn, fn are not confused
print(tn, fp, fn, tp)

In [ ]:
print(classification_report(y_train, dt.predict(X_train))) # yields class-specific precision, recall and f1-score

In [ ]:
print(classification_report(y_test, dt.predict(X_test)))

Performance on test data is significantly lower than on training data. Probably the decision tree overfits on training data and does not generalize well on unseen test data. 

Let's have a look at the feature importance and tree structure. 

In [ ]:
list(zip(X_train.columns, dt.feature_importances_)) # lists features and their importance in predicting the target

In [ ]:
# plot the decision tree
plt.figure(figsize=(30,24))  # set plot size (denoted in inches)
tree.plot_tree(dt, fontsize=5, feature_names=predictors.columns.to_list())# set fontsize
plt.show()
# For better readability you can adjust the figsize and fontsize - or right click on tree --> Open image in new tab

In [ ]:
# alternative 
print(tree.export_text(dt))

### Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, rf.predict(X_test))

In [ ]:
print(classification_report(y_train, rf.predict(X_train)))

In [ ]:
print(classification_report(y_test, rf.predict(X_test)))

As before, test performance is lower than training performance. Random forests, too, can suffer from overfitting on training data. 

In [ ]:
list(zip(X_train.columns, rf.feature_importances_))

### Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
print(confusion_matrix(y_test, logreg.predict(X_test)))

In [ ]:
# nicer way to inspect confusion matrix
conf_mat = confusion_matrix(y_test, logreg.predict(X_test))
df_cm = pd.DataFrame(conf_mat, index=['0','1'], columns=['0', '1'],)
fig = plt.figure(figsize=[10,7])
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_train, logreg.predict(X_train)))

In [ ]:
print(classification_report(y_test, logreg.predict(X_test)))

For logistic regression, training and test performance are very similar. This probably means that the created model generalizes well on new data. 

### Building many models

When building and comparing lots of models, it may be useful to loop over several classifiers or over one classifier with several parameters. An idea to overcome the overfitting problem with tree-based classifiers is to limit the depth of trees and inspect evaluation metrics.

In [ ]:
# vary maximum tree depth for random forest
tree_depth = [5, 10, 20]
for i in tree_depth:
    rf = RandomForestClassifier(max_depth=i)
    rf.fit(X_train, y_train)
    print('Max tree depth: ', i)
    print('Train results: ', classification_report(y_train, rf.predict(X_train)))
    print('Test results: ',classification_report(y_test, rf.predict(X_test)))

Feel free to consider additional aspects if you are familiar with machine learning: You could check for class imbalance and mitigate this by oversampling training data. You could also try more classification algorithms like SVM. 

# Titanic Deployment

In this section you will learn how to deploy a scikit-learn model as a web service with the aid of the _IBM Watson Machine Learning Service_.

## Local Prediction

First, choose the model to deploy and make a local prediction. Later, you will compare the predicted results returned by the local model with the results returned by the web service for evaluation purposes. 

In [ ]:
# assign your favorite model to the deployment_classifier variable
deployment_classifier = logreg
deployment_classifier

In [ ]:
# recap: print first rows of training data
df_dummies.head(2)

In [ ]:
# recap: print first rows of predictors (here: training data without predicted column "Survived")
predictors.head(2)

In [ ]:
# use the local model to make a prediction for the first two passengers
deployment_classifier.predict(predictors.iloc[0:2])

Review the output. Did the model return a correct prediction for the "Survived" field?

## Web service deployment

To work through this section, you need
- a Watson Machine Learning instance
- an IBM Cloud API key
- a deployment space id

Check the guidelines available at https://github.com/ellenhvn/hhz-artificial-intelligence-vl-ws24/tree/main/Guidelines for details.

In [ ]:
# import Python client library (documentation available at http://ibm-wml-api-pyclient.mybluemix.net/)
from ibm_watson_machine_learning import APIClient

In [ ]:
# set your IBM Cloud API key 
api_key = "..."

# set the URL of your WML instance 
# depending on the region you chose during instance creation it will take one of the below values:
# - Frankfurt: https://eu-de.ml.cloud.ibm.com
# - Dallas: https://us-south.ml.cloud.ibm.com
# - London: https://eu-gb.ml.cloud.ibm.com
# - Tokyo: https://jp-tok.ml.cloud.ibm.com
wml_url = "https://us-south.ml.cloud.ibm.com"

In [ ]:
# setup the API client
wml_client = APIClient({
   "url": wml_url,
   "apikey": api_key
})

In [ ]:
# list all existing deployment spaces
wml_client.spaces.list()

In [ ]:
# set the id of the deployment space you want to use as default
wml_client.set.default_space("...")

In [ ]:
# setup required properties to store the model
sofware_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'Titanic Model',
            wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.3',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
metadata

In [ ]:
# store the scikit-learn model in WML
model = wml_client.repository.store_model(deployment_classifier, meta_props=metadata)

In [ ]:
# review available models in your WML instance
wml_client.repository.list()

In [ ]:
# retrieve the id of the model you deployed
published_model_uid = wml_client.repository.get_model_id(model)
published_model_uid

In [ ]:
# setup required properties to deploy the model
metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of Titanic model",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}


In [ ]:
# deploy the model as a web service (an API endpoint is generated for your deployment so your tools and apps can use a REST API to send data to your deployed model for analysis)
created_deployment = wml_client.deployments.create(published_model_uid, name="Titanic Deployment", meta_props=metadata)

In [ ]:
# keep the REST API endpoint for evaluation
scoring_endpoint = wml_client.deployments.get_scoring_href(created_deployment)
scoring_endpoint

Now check the deployment spaces UI (open the menu on the side menu, select "View all spaces" and navigate to the selected space).
- Can you find your model and deployment?
- Can you find code snippets to share with developers that will enable them to make predictions using your web service deployment?

## Evaluate web service deployment

You will now use the REST API (documentation available at https://cloud.ibm.com/apidocs/machine-learning#deployments-compute-predictions) to execute a prediction and compare its results against the local prediction from a previous section.

In [ ]:
# import requests module
import requests

In [ ]:
# setup the request payload as per the API documentation
scoring_values = predictors.iloc[0:2].to_numpy().tolist()
payload_scoring = {"input_data": [{"values": scoring_values}]}

In [ ]:
# create a token to make an authenticated request
token_response = requests.post('https://iam.eu-de.bluemix.net/identity/token', data={"apikey": api_key, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]


In [ ]:
# send the scoring request
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_scoring = requests.post(f'{scoring_endpoint}?version=2022-04-29', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
response_scoring.content

- Do the results match the predictions executed in this notebook?
- What information does the response payload include in addition to the classification?

## Clean up

Free WML instances only allow for a limited number of models and deployments. Let's clean up artefacts that are no longer needed.

In [ ]:
# list deployments
wml_client.deployments.list()

In [ ]:
# delete deployments you just created 
wml_client.deployments.delete("...")

In [ ]:
# list models
wml_client.repository.list_models()

In [ ]:
# delete models you just created
wml_client.repository.delete("...")